In [1]:
import os

In [2]:
%pwd

's:\\OneDrive\\Documents\\Text_Summarizer_AWS_End_To_End\\research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [6]:
import shutil
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

In [7]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size 

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            try:
                filename, headers = request.urlretrieve(
                    url=self.config.source_URL,
                    filename=self.config.local_data_file
                )
                logger.info(f"{filename} downloaded! with following headers:\n{headers}")
            except Exception as e:
                logger.error(f"Error downloading the file: {e}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    # def extract_zip_file(self):
    #     unzip_path = self.config.unzip_dir
    #     try:
    #         os.makedirs(unzip_path, exist_ok=True)
    #         zip_file_path = self.config.local_data_file
    #         shutil.unpack_archive(zip_file_path, unzip_path)
    #         logger.info(f"Zip file {zip_file_path} extracted successfully to {unzip_path}")
    #     except Exception as e:
    #         logger.error(f"Error extracting zip file: {e}")

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    # data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-08-10 05:30:25,249] INFO: common: Yaml file: config\config.yaml loaded sucessfully]
[2023-08-10 05:30:25,258] INFO: common: Yaml file: params.yaml loaded sucessfully]
[2023-08-10 05:30:25,264] INFO: common: Directory: artifacts created sucessfully]
[2023-08-10 05:30:25,265] INFO: common: Directory: artifacts/data_ingestion created sucessfully]
[2023-08-10 05:30:32,588] INFO: 461578954: artifacts/data_ingestion/data.zip downloaded! with following headers:
Connection: close
Content-Length: 23627009
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2e7409b328d118a1d37018be788babf8bf9640386387da766ac100e96efa3b93"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 0824:3529B0:1C4C3F:2735D6:64D4D6BA
Accept-Ranges: bytes
Date: Thu, 10 Aug 2023 12:30:25 GMT
Via: 1.1 varnish
X-Served-By: cache-del